In [255]:
import pandas as pd

In [256]:
import os
from dotenv import load_dotenv

from langchain_google_genai import GoogleGenerativeAI

# import prompt template
from langchain import PromptTemplate

In [257]:
df = pd.read_csv("MASTER_FILES/Master_Final.csv")

/var/folders/nb/6qvvb9dj30l_kxkvsvmsq25m0000gn/T/ipykernel_3532/1118296231.py:1: DtypeWarning: Columns (0,2,3,5,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("MASTER_FILES/Master_Final.csv")


In [258]:
print(df.columns)

Index(['Scheduled_Date', 'Brand', 'City', 'Ad_Lang', 'Amount', 'ActivityName',
       'Medium', 'State', 'Market', 'PO_Date', 'CreatedBy', 'Agency',
       'ApprovedBy', 'Spot_Type', 'Parent Network', 'Channel', 'Caption'],
      dtype='object')


In [259]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path='/Users/bhumika/Downloads/venv/bi_chatbot/.env')

# Set environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "pr-timely-slaw-83"
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [260]:
llm = GoogleGenerativeAI(model="gemini-pro",temperature=0.1)
llm.invoke("Who are you? and what can you do?")

I0000 00:00:1724149505.366894   62965 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


'I am Gemini, a multimodal AI language model developed by Google. I am designed to understand and generate human language, answer questions, and provide information on a wide range of topics.\n\nHere are some of the things I can do:\n\n* **Answer questions:** I can answer questions on a wide range of topics, including history, science, current events, and more.\n* **Provide information:** I can provide information on a wide range of topics, including weather, sports, stocks, and more.\n* **Generate text:** I can generate text in a variety of styles, including news articles, stories, poems, and more.\n* **Translate languages:** I can translate text between over 100 languages.\n* **Summarize text:** I can summarize text into a shorter, more concise version.\n* **Answer questions in a conversational way:** I can answer questions in a conversational way, making it easy to interact with me.\n\nI am still under development, but I am learning new things every day. I am excited to see how I ca

In [102]:

# create the prompt-1
prompt_template: str = """

You are a query enhancing agent. You have to work with the query provided by the user : {query}.
The user wants to inquire about information from the ad campaigns dataset, which contain the spending
and other information of different mediums which are represented by Medium column, namely - TV , Print , Radio ,  Digital, OOH and BTL.
The dataset contains following columns and description -

"Brand": "The brand associated with the ad campaign.",
"Agency": "The advertising agency managing the ad campaign.",
"CreatedBy": "The individual or entity who created the document.",
"PO_Date": "The date the purchase order was created.",
"Market": "The market targeted by the ad campaign.",
"Channel": "The channel through which the ad is broadcasted or published",
"Parent Network": "The parent network associated with the channel.",
"Caption": "The caption or title of the ad.",
"Ad_Lang": "The language in which the ad is presented.",
"Scheduled_Date": "The scheduled date for the ad campaign.",
"Amount": "The total gross amount spent on the ad campaign.",
"ApprovedBy": "The individual or entity who approved the document.",
"Spot_Type": "The type of ad spot (e.g., prime time, regular, etc.).",
"Medium": "The medium of the ad campaign (e.g., TV, radio, digital, print, OOH, BTL).",
"State": "The state where the ad campaign is being targeted.",
"City": "The city where the ad campaign is being targeted.",
'ActivityName': 'The name or title of the specific ad campaign activity.'


Now you have to understand the query and see what are the relevant columns that would be required to solve the query.
Then using those relevant columns enhance the query so that there is no key value error when that query is used to generate code.

Return your answer in dictionary format as below -

"Enhanced Question" : the updated query,
"Relevant Columns": relevant columns in a list
"""

prompt = PromptTemplate.from_template(template=prompt_template)

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    query="What was the spend for Radio Mirchi Jaipur in June month?")

answer = llm.invoke(prompt_formatted_str)

import ast
# Convert the string to a dictionary
def convert_string_to_dict(input_string):
    # Use ast.literal_eval to safely evaluate the string as a dictionary
    dictionary = ast.literal_eval(input_string)
    return dictionary

# Call the function and print the result
result_dict = convert_string_to_dict(answer)
enhanced_question = result_dict['Enhanced Question']
relevant_columns = result_dict['Relevant Columns']

print(result_dict)


{'Enhanced Question': 'What was the total spending on Radio Mirchi in Jaipur during June?', 'Relevant Columns': ['Medium', 'State', 'City', 'Amount', 'Scheduled_Date']}


In [103]:

# create the prompt-2
prompt_template: str = """

You are a query enhancing agent. You have to work with the query provided by the user : {query}.
The user wants to inquire about information from the ad campaigns dataset, which contain the spending
and other information of different mediums which are represented by Medium column.
The dataset contains following columns and description -

"Brand": "The brand associated with the ad campaign.",
"Agency": "The advertising agency managing the ad campaign.",
"CreatedBy": "The individual or entity who created the document.",
"PO_Date": "The date the purchase order was created.",
"Market": "The market targeted by the ad campaign.",
"Channel": "This column may include the name of a TV channel, radio station, newspaper, or social media platform through which the ad is broadcasted or published.",
"Parent Network": "The parent network associated with the channel.",
"Caption": "The caption or title of the ad.",
"Ad_Lang": "The language in which the ad is presented.",
"Scheduled_Date": "The scheduled date for the ad campaign.",
"Amount": "The total gross amount spent on the ad campaign.",
"ApprovedBy": "The individual or entity who approved the document.",
"Spot_Type": "The type of ad spot (e.g., prime time, regular, etc.).",
"Medium": "The medium of the ad campaign (e.g., TV, radio, digital, print, OOH, BTL).",
"State": "The state where the ad campaign is being targeted.",
"City": "The city where the ad campaign is being targeted.",
'ActivityName': 'The name or title of the specific ad campaign activity.'


You need to analyze the query and, using general knowledge and the descriptions provided for each column, examine the columns to determine which ones are relevant to solve the query.

For example, if the user query is "In what languages is Lokmat newspaper available?"
The required columns would be "Ad_Lang" for languages and "Channel" or "Parent Network"
for Lokmat newspaper.

Another example, if the user query is "What was the spend for Jaipur West for Radio Mirchi in June month?"
The required columns would be "Amount" for spend, "State" for Jaipur, "Market" for West, "Channel" or "Parent Network" for Radio Mirchi, and "Scheduled_Date" for June.

Therefore, you need to analyze every word in the user query to determine which columns are necessary from the given options.

Return your answer in dictionary format as below -
"Relevant Columns": relevant columns in a list
"""

prompt = PromptTemplate.from_template(template=prompt_template)

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    query="What was the spend for Jaipur West for Radio Mirchi in June month?")

answer = llm.invoke(prompt_formatted_str)

import ast
# Convert the string to a dictionary
def convert_string_to_dict(input_string):
    # Use ast.literal_eval to safely evaluate the string as a dictionary
    dictionary = ast.literal_eval(input_string)
    return dictionary

# Call the function and print the result
result_dict = convert_string_to_dict(answer)
relevant_columns = result_dict['Relevant Columns']

print(result_dict)

{'Relevant Columns': ['Amount', 'State', 'Market', 'Channel', 'Scheduled_Date']}


In [180]:
# create the prompt-2
prompt_template: str = """

You are a query enhancing agent. You have to work with the query provided by the user : {query}.
The user wants to inquire about information from the ad campaigns dataset, which contain the spending
and other information of different mediums which are represented by Medium column.
The dataset contains following columns and description -

"Brand": "The brand associated with the ad campaign.",
"Agency": "The advertising agency managing the ad campaign.",
"CreatedBy": "The individual or entity who created the document.",
"PO_Date": "The date the purchase order was created.",
"Market": "The market targeted by the ad campaign.",
"Channel": "This column may include the name of a TV channel, radio station, newspaper, or social media platform through which the ad is broadcasted or published.",
"Parent Network": "The parent network associated with the channel.",
"Caption": "The caption or title of the ad.",
"Ad_Lang": "The language in which the ad is presented.",
"Scheduled_Date": "The scheduled date for the ad campaign.",
"Amount": "The total gross amount spent on the ad campaign.",
"ApprovedBy": "The individual or entity who approved the document.",
"Spot_Type": "The type of ad spot (e.g., prime time, regular, etc.).",
"Medium": "The medium of the ad campaign (e.g., TV, radio, digital, print, OOH, BTL).",
"State": "The state where the ad campaign is being targeted.",
"City": "The city where the ad campaign is being targeted.",
'ActivityName': 'The name or title of the specific ad campaign activity.'


You need to analyze the query and, using general knowledge and the descriptions provided for each column, examine the columns to determine which ones are relevant to solve the query.

For example, if the user query is "In what languages is Lokmat newspaper available?"
The required columns would be "Ad_Lang" for languages and "Channel" or "Parent Network"
for Lokmat newspaper.

Another example, if the user query is "What was the spend for Jaipur West for Radio Mirchi in June month?"
The required columns would be "Amount" for spend, "State" for Jaipur, "Market" for West, "Channel" or "Parent Network" for Radio Mirchi, and "Scheduled_Date" for June.

Therefore, you need to analyze every word in the user query to determine which columns are necessary from the given options.

Return your answer in dictionary format as below -
"Relevant Columns": relevant columns in a list
"""

prompt = PromptTemplate.from_template(template=prompt_template)

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    query="In what languages is EENADU Newspaper available")

answer = llm.invoke(prompt_formatted_str)

import ast
# Convert the string to a dictionary
def convert_string_to_dict(input_string):
    # Use ast.literal_eval to safely evaluate the string as a dictionary
    dictionary = ast.literal_eval(input_string)
    return dictionary

# Call the function and print the result
result_dict = convert_string_to_dict(answer)
relevant_columns = result_dict['Relevant Columns']

print(result_dict)

{'Relevant Columns': ['Channel', 'Ad_Lang']}


In [318]:
# create the prompt-3
prompt_template: str = """

You are a query enhancing agent. You have to work with the query provided by the user : {query}.
The user wants to inquire about information from the ad campaigns dataset, which contain the spending
and other information of different mediums which are represented by Medium column.
The dataset contains following columns and description -

"Brand": "The brand associated with the ad campaign.",
"Agency": "The advertising agency managing the ad campaign.",
"CreatedBy": "The individual or entity who created the document.",
"PO_Date": "The date the purchase order was created.",
"Market": "The market targeted by the ad campaign.",
"Channel": "This column may include the name of a TV channel, radio station, newspaper, or social media platform through which the ad is broadcasted or published.",
"Parent Network": "The parent network associated with the channel.",
"Caption": "The caption or title of the ad.",
"Ad_Lang": "The language in which the ad is presented.",
"Scheduled_Date": "The scheduled date for the ad campaign.",
"Amount": "The total gross amount spent on the ad campaign.",
"ApprovedBy": "The individual or entity who approved the document.",
"Spot_Type": "The type of ad spot (e.g., prime time, regular, etc.).",
"Medium": "The medium of the ad campaign (e.g., TV, radio, digital, print, OOH, BTL).",
"State": "The state where the ad campaign is being targeted.",
"City": "The city where the ad campaign is being targeted.",
'ActivityName': 'The name or title of the specific ad campaign activity.'


You need to analyze the query and, using general knowledge and the descriptions provided for each column, examine the columns to determine which ones are relevant to solve the query.

For example, if the user query is "In what languages is Lokmat newspaper available?"
The required columns would be "Ad_Lang" for languages and "Channel" or "Parent Network"
for Lokmat newspaper.

Another example, if the user query is "What was the spend for Jaipur West for Radio Mirchi in June month?"
The required columns would be "Amount" for spend, "State" for Jaipur, "Market" for West, "Channel" or "Parent Network" for Radio Mirchi, and "Scheduled_Date" for June.

Therefore, you need to analyze every word in the user query to determine which columns are necessary from the given options.
Then using those relevant columns enhance the query so that there is no key value error when that query is used to generate code.

Return your answer in dictionary format as below -
"Enhanced Question" : the updated query,
"Relevant Columns": relevant columns in a list
"""

prompt = PromptTemplate.from_template(template=prompt_template)

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    query="What is the Market for top spent Channel?.")

answer = llm.invoke(prompt_formatted_str)

import ast
# Convert the string to a dictionary
def convert_string_to_dict(input_string):
    # Use ast.literal_eval to safely evaluate the string as a dictionary
    dictionary = ast.literal_eval(input_string)
    return dictionary

# Call the function and print the result
result_dict = convert_string_to_dict(answer)
enhanced_question = result_dict['Enhanced Question']
relevant_columns = result_dict['Relevant Columns']

print(result_dict)

{'Enhanced Question': 'What is the Market for the top spent Channel?', 'Relevant Columns': ['Market', 'Channel', 'Amount']}


In [319]:
categorical_columns = ['Brand', 'Agency', 'CreatedBy', 'Market', 'Channel',
                        'Parent Network', 'Caption', 'Ad_Lang','ApprovedBy',
                        'Spot_Type', 'Medium', 'State', 'City']

# Identify categorical columns in the relevant columns
categorical_relevant_columns = [col for col in relevant_columns if col in categorical_columns]

# Get unique values for categorical columns
unique_values = {}
for col in categorical_relevant_columns:
    unique_values[col] = df[col].unique()



### Prompt Generation

In [320]:
# df_description = '''
# You are a data analysis agent specializing in helping non-technical users understand and interpret data accurately.
# Your main objective is to provide clear and concise answers to user's questions in the form of strings only, without generating any graphs or visual aids.
# Your role involves interpreting user's queries, which may include spelling or capitalization errors, and using the closest matching correct values from the relevant column and rows to ensure accurate analysis.
# The data you work with is a single dataframe representing ad campaigns, which includes various relevant columns.
# One key column is Medium, which represents the type of medium in which the ad campaign was displayed, which consists of Print, TV (television), Radio, Digital, OOH (outdoor), and BTL (below the line marketing).
# This data is related to a user who has spent money on ad campaigns across different brands, indicated by the column called Brand, which shows the brand for which the money was spent.
# The dataframe also includes a column called Amount that indicates the amount spent on the campaign.
# Additionally, there is a column called CreatedBy, showing the name of the person who created the campaign, and a column called ApprovedBy, showing the name of the person who approved the campaign.
# If a user queries information specifying the name of a person, use the CreatedBy column if the focus is on creation, and the ApprovedBy column if the focus is on approval.
# If a user queries information by specifying mediums, you should use the available rows for the specified mediums only to provide a comprehensive response.
# If the name of a channel or network is mentioned in the query, use the Parent Network column for networks or the Channel column for channels to get relevant information.
# For the Digital medium, the Parent Network and Channel columns consist of social media platforms, websites, news channels, and other organizations that operate digitally.
# For the Print medium, these columns consist of newspaper publications.
# Additionally, if a user mentions a city, state, or market, you should refer to the respective columns called —City, State, or Market.
# For questions about spends related to parent networks or channels, ensure that the response is not zero; there must be some spends, so further investigation may be necessary.
# '''

In [321]:
df_description = '''

The data is stored in Clickhouse Database named "default" inside table named "campaigns".
Table "data" consists of 17 columns. They are as follows (column_name, data_type, description):
    "Medium", "String", "The medium where the ad campaign takes place. It consists of 6 values: TV, Radio, Print, Digital, Outdoor and BTL."
	"PO_Date", "Date", "The date of creation of the ad campaign. It is Null for the medium: BTL."
	"Scheduled_Date", "Date", "The date at which the ad campaign is displayed. It is Null for the medium: Outdoor."
	"CreatedBy", "String", "Name of the person who created the ad campaign. It is Null for the medium: BTL."
 	"ApprovedBy", "String", "Name of the person who approved the ad campaign. It is Null for the medium: BTL."
	"Brand", "String", "The various brand names for which the ad campaign takes place."
	"Agency", "String", "The agency or company via which the ad campaign takes place. It is Null for the medium: BTL."
 	"Channel", "String", "The channel on which the ad campaign is displayed. For medium: TV, the channel refers to TV Channels; for medium: Radio, the channel refers to Radio Channels; for medium: Print, the channel refers to Newspaper Names; for medium: Digital, the channel refers to online channels like google or facebook; for medium: Outdoor and BTL, the channel is Null."
  	"Parent Network", "String", "The parent network of the channel. There may be many channels under a single parent network. For medium: Print, the parent network refers to the Publication Company of the Newspaper. It is Null for the medium: Outdoor and BTL."
	"City", "String", "Name of the city. It is Null for the medium: Digital and TV."
	"State", "String", "Name of the state. It is Null for the medium: Digital and TV."
	"Market", "String", "Region of the ad campaign. Has 7 values: North, South, East, West, Central, All India and NaN."
	"Ad_Lang", "String", "Language of the ad. It is Null for the medium: Digital."
	"Caption", "String", "Caption of the ad campain. It is Null for the medium: Digital and BTL."
	"ActivityName", "String", "It is Null for all mediums except BTL."
 	"Spot_Type", "String", "It consists of two vales: SPOT and IMPACT. It is Null for the medium: Digital, Outdoor and BTL."	
 	"Amount", "Float64", "The amount spend on the ad campaign." 
'''

In [322]:
import random

def generate_prompt(df_description, df, enhanced_question, relevant_columns, unique_values):
    # Initialize the prompt with a description of the dataset
    prompt = df_description

    # Add random samples from the dataset for each column
    prompt += "\n\n### Dataset Samples\n"
    for col in df.columns:
        # Get random samples from the column, ignoring NaN values
        samples = df[col].dropna().sample(n=3, random_state=1).tolist()
        samples_str = "\n      - ".join(map(str, samples))
        prompt += f"- name: {col}\n  type: {df[col].dtype}\n  samples:\n      - {samples_str}\n"

    # Add the enhanced user query
    prompt += "\n### Enhanced User Query\n"
    prompt += f"{enhanced_question}\n"

    # Add relevant columns
    prompt += "\n### Relevant Columns\n"
    prompt += f"{', '.join(relevant_columns)}\n"

    # Add unique values for relevant columns
    prompt += "\n### Unique Values\n"
    for col in relevant_columns:
        prompt += f"- {col}: {', '.join(map(str, unique_values.get(col, [])))}\n"


    return prompt


# Generate the prompt
prompt = generate_prompt(df_description, df, enhanced_question, relevant_columns, unique_values)
print(prompt)




The data is stored in Clickhouse Database named "default" inside table named "campaigns".
Table "data" consists of 17 columns. They are as follows (column_name, data_type, description):
    "Medium", "String", "The medium where the ad campaign takes place. It consists of 6 values: TV, Radio, Print, Digital, Outdoor and BTL."
	"PO_Date", "Date", "The date of creation of the ad campaign. It is Null for the medium: BTL."
	"Scheduled_Date", "Date", "The date at which the ad campaign is displayed. It is Null for the medium: Outdoor."
	"CreatedBy", "String", "Name of the person who created the ad campaign. It is Null for the medium: BTL."
 	"ApprovedBy", "String", "Name of the person who approved the ad campaign. It is Null for the medium: BTL."
	"Brand", "String", "The various brand names for which the ad campaign takes place."
	"Agency", "String", "The agency or company via which the ad campaign takes place. It is Null for the medium: BTL."
 	"Channel", "String", "The channel on which th

In [323]:
import clickhouse_connect

In [324]:
client = clickhouse_connect.get_client(host="localhost", port=8123)

In [325]:
query = """SELECT name AS column_name, type AS data_type FROM system.columns WHERE table = 'campaigns' AND database = 'default'"""

result = client.query(query)

print(result)

In [326]:
result_df = client.query_df(query)


In [327]:
result_df['column_name'].to_list()

['Scheduled_Date',
 'Brand',
 'City',
 'Ad_Lang',
 'Amount',
 'ActivityName',
 'Medium',
 'State',
 'Market',
 'PO_Date',
 'CreatedBy',
 'Agency',
 'ApprovedBy',
 'Spot_Type',
 'Parent Network',
 'Channel',
 'Caption']

In [328]:
# Query to get the total number of rows in the table
total_rows_query = "SELECT COUNT(*) AS total_rows FROM campaigns"
total_rows_result = client.query(total_rows_query)

# Extract the total row count from the result
total_rows = total_rows_result.result_rows[0][0]  # Assuming result_rows returns a list of tuples
print(f"Total number of rows: {total_rows}")

Total number of rows: 1787621


In [329]:
sql_prompt = """
Using the information provided below, generate a precise SQL query to be executed on the 'campaigns' table in ClickHouse. Ensure the query:
1. Retrieves only distinct values where applicable.
2. Utilizes the relevant columns specified.
3. Avoids any key errors by using provided unique values.
4. Correctly reflects the user’s query by focusing on the specific requirements mentioned.
{}
""".format(prompt)

# Request SQL code generation from the AI model
sql_code = llm.invoke(sql_prompt)

# Print the generated SQL code
print("Generated SQL Code:\n", sql_code)



Generated SQL Code:
 ```sql
SELECT
  Market,
  Channel,
  SUM(Amount) AS TotalAmount
FROM campaigns
GROUP BY
  Market,
  Channel
ORDER BY
  TotalAmount DESC
LIMIT 1;
```


In [330]:
# Clean up the SQL code by removing unnecessary formatting
import re

def clean_sql_code(sql_code):
    # Remove backticks and any extra characters
    cleaned_code = re.sub(r'```sql|```', '', sql_code).strip()
    return cleaned_code

# Clean the SQL code
cleaned_sql_code = clean_sql_code(sql_code)

print("Cleaned SQL Code:\n", cleaned_sql_code)

Cleaned SQL Code:
 SELECT
  Market,
  Channel,
  SUM(Amount) AS TotalAmount
FROM campaigns
GROUP BY
  Market,
  Channel
ORDER BY
  TotalAmount DESC
LIMIT 1;


In [331]:
try:
    result = client.query(cleaned_sql_code)
    print("Query Result:\n", result)
except Exception as e:
    print("An error occurred while executing the SQL query:", str(e))

Query Result:


In [332]:
import json
result_data = result.result_rows
result_json = json.dumps(result_data)
print(result_json)

[["North", null, 40648499795.36029]]


In [333]:
# Create a prompt for the LLM to frame the answer
answer_prompt_template = """
You are an assistant that provides clear and concise answers based on query results.

The SQL query executed was:
{sql_code}

The result obtained from the database is:
{result}

Please frame an answer to present this result in a user-friendly manner, explaining the information in simple terms and addressing the user's query based on the result provided.
"""

answer_prompt = answer_prompt_template.format(sql_code=cleaned_sql_code, result=result_json)

# Request the answer framing from the LLM
try:
    answer = llm.invoke(answer_prompt)
    print("Framed Answer:\n", answer)
except Exception as e:
    print("An error occurred while generating the answer:", str(e))


Framed Answer:
 The top-performing combination of market and channel in terms of total amount is "North" market with no specific channel specified. This combination has generated a total amount of $40,648,499,795.36.
